# Alineamiento de datos

### LEE ATENTAMENTE ESTE `notebook` EJECUTANDO SUS CELDAS. COMPLETA Y CONTESTA LAS CELDAS QUE CORRESPONDAN

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

!echo "HOSTNAME     " `hostname`; 
!echo "CURRENT DIR  " `pwd`

## Instrucciones
Inspecciona y entiende el siguiente programa en C. Observa como:
* la función `ji` recorre un array de dos dimensiones modificando cada elemento
* la función `main` crea arrays de distintos tamaños y, para cada tamaño, ejecuta varias veces las funciones `ij` y `ji` y promedia los tiempos de ejecución de cada una
* los tiempos promedios de ejecución de cada función se escriben en un fichero y también en `stdout`


### ES RECOMENDABLE QUE EJECUTES LAS CELDAS DE ESTE NOTEBOOK ANTES EMPEZAR TU TAREA
tratando de entender qué está haciendo el código y cómo interpretar los datos obtenidos

# TAREA: Programa la función `ij` en C
de forma idéntica a la función `ji` pero intercambiando el orden de los bucles `for` (haciendo un `for` sobre la variable `j` dentro de un `for` sobre la variable `i`)

**$\rightarrow$ modifica el código y sálvalo ** (`shift` + `return`)

In [ ]:
%%writefile files/access_order_example.c
#include <stdlib.h>
#include "rlxutils.h"

void ji(char *arr, long SIZE) {
    long i,j;
    for (j=0; j<SIZE; j++) {
     for (i=0; i<SIZE; i++) {
         arr[i+SIZE*j]++;
     }
    }
}

void ij(char *arr, long SIZE) {
    //  =========== MODIFICA DESDE AQUI ============
    //  recorre el array en orden inverso a la funcion ij
    //  aumentando el valor de cada elemento en una unidad
    // =========== MODIFICA HASTA AQUI ============
}

int main(int argc, char **argv) {
    struct timespec tsi, tsf;
    long N = 10;  
    long i,j,k,s;
    double time_diff;

    char *fname = "files/Carrays.data";
    printf ("writing to file %s and stdout\n", fname);
    FILE *f = fopen(fname, "w");
    if (f == NULL)
    {
        printf("Error opening file %s\n",fname);
        exit(1);
    }

    double time_ij[N];
    double time_ji[N];

    long sizes[] = {8, 16, 32, 64, 128, 384, 512, 640, 768, 1024, 1536, 2048, 2560,3072, 3584, 4096, 4608, 5120};

    long ssizes = sizeof(sizes)/sizeof(long);
    char *header = "iteration array_size elapsed_time_ij elapsed_time_ji\n";
    printf(    "%s", header);
    fprintf(f, "%s", header);

    for (s=0; s<ssizes; s++) {
      long array_size = sizes[s];
      for (k=0; k<N; k++) {

          char *arr = malloc(array_size*array_size*sizeof(char));
          current_utc_time(&tsi);
          ij(arr, array_size);
          current_utc_time(&tsf);
          time_ij[k] = get_time_diff(tsi, tsf);

          current_utc_time(&tsi);
          ji(arr, array_size);
          current_utc_time(&tsf);
          time_ji[k] = get_time_diff(tsi, tsf);

          free(arr);
      }
      double total_array_size = power(sizes[s],2);
      printf(    "%lu %3.10f %lf %lf\n",s, total_array_size/1024, avg(time_ij, N), avg(time_ji,N));
      fprintf(f, "%lu %3.10f %lf %lf\n",s, total_array_size/1024, avg(time_ij, N), avg(time_ji,N));

    }
    fclose(f);
    return 0; 

}

### Compila tu código

**$\rightarrow$ ejecuta** (`shift` + `return`)

In [ ]:
!make -f m/Makefile-p1.1 clean all

### Ejecuta tu código

**$\rightarrow$ ejecuta** (`shift` + `return`)
<br/>y espera a que acabe la ejecución, observa el formato de los datos de salida

In [ ]:
!./files/access_order_example 

### Carga los datos generados por el programa en C en un `DataFrame` de Python

**$\rightarrow$ ejecuta** (`shift` + `return`)

In [ ]:
file_name = "files/Carrays.data"
df = pd.read_csv(file_name, sep=" ")

it_number = df['iteration']
x_labels  = df['array_size']

df

### Pinta los datos
observa cómo hacemos el eje horizontal aparecer como una escala no lineal con el tamaño total del array generado $(N^2)$

**$\rightarrow$ ejecuta** (`shift` + `return`)

In [ ]:
ax = plt.figure(figsize=(13,5)).add_subplot(111)
plots = []
legends = []
for k in df.keys()[2:]:
    p, = ax.plot(it_number,df[k], linewidth=5, alpha=0.5)
    plots = plots + [p]
    legends = legends+[k]

ax.set_xticks(it_number)
ax.legend(plots, legends, loc=2)
ax.set_xticklabels([str(int(i)) for i in x_labels])
ax.set_xlabel("total array size (KB)")
ax.set_ylabel("secs")

# PREGUNTAS:
* ¿En qué orden crees más conveniente recorrer el array, primero por filas o primero por columnas?
* ¿Por qué? ¿Cómo explicas las diferencias de tiempos y de comportamiento?

--- TU RESPUESTA AQUí (double click) ---